In [1]:
import ee
ee.Initialize()
import requests
import os
import zipfile
#RnkUmmi -52.89 71.55 -51.43 71.82

The process to authenticate Earth Engine

In [2]:
#Creates a geometry object with cordinates of RnkUmmi
geometry = ee.Geometry.Rectangle([-52.89, 71.55, -51.43, 71.82])

#variables that Jonathan had
data_product = 'LANDSAT/LE07/C01/T1'
sel_bands = ['B5','B4','B3']

In [3]:
#creates a temp folder to download the images to
save_folder = 'RnkUmmi_Landsat-7'
os.makedirs(save_folder, exist_ok=True)

#For loops that traverses all the years sequentially
for year in range(1999,2013):
    start = ee.Date(f'{year}-07-20')
    end = ee.Date(f'{year}-09-10')

    #This is a imageCollection object that has all the filters applied
    #Creates a new collection everytime there is a new year 
    filteredCollection = ee.ImageCollection(data_product).filterBounds(geometry).filterDate(start, end).select(sel_bands).filterMetadata('CLOUD_COVER', 'less_than', 30)

    #Why is this list created? Its an object that holds pictures. Just need the pictures
    filtered_list=filteredCollection.toList(filteredCollection.size())
    #gets the count of the list 
    filteredCount = filtered_list.size().getInfo()
    print("Filtered count is %d"%(filteredCount))
    print(f"year is {year}, {start.format('YYYY-MM-dd').getInfo()} and {end.format('YYYY-MM-dd').getInfo()}")
    
    #for loop that traveses all the pictures in the list
    for i in range(filteredCount):
        #Make sure the list are inside the geometry object
        image = ee.Image(filtered_list.get(i)).clip(geometry)
        #Prints and formats the name of the GeoTIFF that is being downloaded
        fname = image.get('system:id').getInfo().split('/')[-1]
        print("downloading %s"%fname)
        try:
            #creates the download url for the image
            url = image.getDownloadURL()

            #makes a request to download the image
            r=requests.get(url)
        except Exception as e:
            print("Failed to get Download URL")
            print(url)

        #names each file 
        filename = f"data_{i}.zip"
        filepath = os.path.join(save_folder, filename)
        print(f"Downloading to: {filepath}, {i} in Filtered Count")
        try:
            #Download and saves the image
            with open(filepath, 'wb') as f:
                f.write(r.content)
        except Exception as e:
            print(f"Error failure to download")
        try:
            with zipfile.ZipFile(filepath) as f:
                files = f.namelist()
                f.extractall(save_folder)
            os.remove(filepath)
            print(f"Downloaded {filename}")
            print(f'Extracted {files} tif files from {filename}')
        except zipfile.BadZipFile:
            print('Skipping - Not a valid ZIP file')
        except Exception as e:
            print('Error extracting')


print("Downloading complete")
#Downloads the data locally # name='./'+save_folder+'/'+'data.zip'
# path='./'+save_folder


Filtered count is 4
year is 1999, 1999-07-20 and 1999-09-10
downloading LE07_011010_19990801
Downloaded data_0.zip
Extracted ['LE07_011010_19990801.B5.tif', 'LE07_011010_19990801.B4.tif', 'LE07_011010_19990801.B3.tif'] tif files from data_0.zip
downloading LE07_012009_19990824
Downloaded data_1.zip
Extracted ['LE07_012009_19990824.B5.tif', 'LE07_012009_19990824.B4.tif', 'LE07_012009_19990824.B3.tif'] tif files from data_1.zip
downloading LE07_012010_19990824
Downloaded data_2.zip
Extracted ['LE07_012010_19990824.B5.tif', 'LE07_012010_19990824.B4.tif', 'LE07_012010_19990824.B3.tif'] tif files from data_2.zip
downloading LE07_013009_19990831
Downloaded data_3.zip
Extracted ['LE07_013009_19990831.B5.tif', 'LE07_013009_19990831.B4.tif', 'LE07_013009_19990831.B3.tif'] tif files from data_3.zip
Filtered count is 5
year is 2000, 2000-07-20 and 2000-09-10
downloading LE07_012009_20000810
Downloaded data_0.zip
Extracted ['LE07_012009_20000810.B5.tif', 'LE07_012009_20000810.B4.tif', 'LE07_012009